In [12]:
# Import data analysis and visualisation packages.
import numpy as np
import pandas as pd
import stan as ps
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import arviz as az
import os
from patsy import dmatrix
from linearmodels.iv import IV2SLS

# Importing nest_asyncio is only necessary to run pystan in Jupyter Notebooks.
import nest_asyncio
nest_asyncio.apply()

#Specify the number of chains to the number of availible cpu's.
n_chains = os.cpu_count()
n_samples = 500
#Convert to int so Stan will not crash below.
n_warmup = int(n_samples/2)
#Specify step size.
stepS = .8

In [2]:
def read_data(file): 
    return pd.read_stata("https://github.com/scunning1975/mixtape/raw/master/"  + file)

card = read_data("card.dta")
card = card[['lwage', 'exper','black', 'south', 'married', 'smsa','educ', 'nearc4']]
cardNNA = card.dropna(axis=0)
cardNNA['exper_sd'] = (cardNNA['exper']-cardNNA['exper'].mean())/cardNNA['exper'].std()
cardNNA['lwage_sd'] = (cardNNA['lwage']-cardNNA['lwage'].mean())/cardNNA['lwage'].std()
cardNNA['educ_sd'] = (cardNNA['educ']-cardNNA['educ'].mean())/cardNNA['educ'].std()

/tmp/ipykernel_5248/3123229734.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardNNA['exper_sd'] = (cardNNA['exper']-cardNNA['exper'].mean())/cardNNA['exper'].std()
/tmp/ipykernel_5248/3123229734.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardNNA['lwage_sd'] = (cardNNA['lwage']-cardNNA['lwage'].mean())/cardNNA['lwage'].std()
/tmp/ipykernel_5248/3123229734.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [14]:
IVMod = '''
data{
int<lower = 1> N;
int<lower = 1> K;
// Number of predicitors in the regressions
int<lower = 1> K1;
int<lower = 1> K2;

# DVs
array[N] row_vector[K]  y;

# IVs
matrix[N, K1] X1;
matrix[N, K2] X2;
}
parameters{
// Regression parameters
vector[K1] beta1;  
vector[K2] beta2;  
vector<lower =0>[K] sigma;
// Correlation matrix
cholesky_factor_corr[K] rho;

}
transformed parameters{
// generate mu for each linear model
vector[N] mu1 = X1 * beta1;
vector[N] mu2 = X2 * beta2;

// generate a tempary matrix varaible
matrix[N, K] mu_temp = append_col(mu1, mu2);

// Convert that matrix into vectors for use by mulit-normal dist
// below in the likelihood
array[N] row_vector[K]  mu;
for (n in 1:N)
    mu[n] = mu_temp[n];
    
matrix[K,K] Sigma = diag_pre_multiply(sigma, rho);
}

model{
// Priors
sigma ~ exponential(1);


// Uniform prior for correlation parameters
rho ~ lkj_corr_cholesky(1);


// linear model priors
beta1 ~ std_normal();
beta2 ~ std_normal();

// Likelihood
y ~ multi_normal_cholesky(mu, Sigma);
}
generated quantities{
//corr_matrix[K] cor_mat = Sigma * Sigma'; 
}
'''

In [1]:
x1 = np.asarray(dmatrix('~1 + exper_sd + black + south + married + smsa', data = cardNNA))
x2 = np.asarray(dmatrix(' ~ nearc4', data = cardNNA))
y = np.asarray(cardNNA[['lwage_sd', 'educ_sd']])
y

NameError: name 'np' is not defined

In [16]:
d ={'N': len(cardNNA),
    'y': y,
    'K': np.shape(y)[1],
    'K1': np.shape(x1)[1],
    'K2': np.shape(x2)[1],
    'X1': x1,
    'X2': x2
}

In [17]:
sm = ps.build(IVMod, data = d)

Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 

In file included from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor.hpp:12,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/rev/fun.hpp:198,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/harrison/.cache/httpstan/4.6.1/models/mffguglv/model_mffguglv.cpp:2:
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor/hcubature.hpp: In function 'void stan::math::internal::combination(std::vector<int>&, const int&, const int&, const int

Building: 8.2s
Building: 8.3s
Building: 8.4s
Building: 8.6s


In file included from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/multi_array/multi_array_ref.hpp:32,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/multi_array.hpp:34,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/numeric/odeint/algebra/multi_array_algebra.hpp:22,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/numeric/odeint.hpp:63,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor/ode_rk45.hpp:9,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor/integrate_ode_rk45.hpp:6,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/inc

Building: 8.7s
Building: 8.8s
Building: 8.9s
Building: 9.0s
Building: 9.1s
Building: 9.2s
Building: 9.3s
Building: 9.4s
Building: 9.5s
Building: 9.6s
Building: 9.7s
Building: 9.8s
Building: 9.9s
Building: 10.0s
Building: 10.1s
Building: 10.2s
Building: 10.3s
Building: 10.4s
Building: 10.5s
Building: 10.6s
Building: 10.7s


...

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/elementwise_check.hpp: In instantiation of 'void stan::math::elementwise_check(const F&, const char*, const char*, const T&, const char*, const Indexings& ...) [with F = check_finite<Eigen::Matrix<double, 2, 1> >(const char*, const char*, const Eigen::Matrix<double, 2, 1>&)::<lambda(double)>; T = Eigen::Matrix<double, 2, 1>; Indexings = {}; stan::require_eigen_t<S>* <anonymous> = 0; std::enable_if_t<((Eigen::internal::traits<_Rhs>::Flags & Eigen::LinearAccessBit) || T::IsVectorAtCompileTime)>* <anonymous> = 0]':
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/check_finite.hpp:29:20:   required from 'void stan::math::check_finite(const char*, const char*, const T_y&) [with T_y = Eigen::Matrix<double, 2, 1>]'
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/fun/hypergeometri

Building: 12.2s
Building: 12.3s
Building: 12.4s
Building: 12.5s
Building: 12.6s
Building: 12.7s
Building: 12.8s
Building: 12.9s
Building: 13.0s
Building: 13.2s
Building: 13.3s
Building: 13.4s
Building: 13.5s
Building: 13.6s
Building: 13.7s
Building: 13.8s
Building: 13.9s
Building: 14.0s
Building: 14.1s
Building: 14.2s
Building: 14.3s
Building: 14.4s
Building: 14.5s
Building: 14.6s
Building: 14.7s
Building: 14.8s
Building: 14.9s
Building: 15.0s
Building: 15.1s
Building: 15.2s
Building: 15.3s
Building: 15.4s
Building: 15.5s
Building: 15.6s
Building: 15.7s
Building: 15.8s
Building: 15.9s
Building: 16.0s
Building: 16.1s
Building: 16.2s
Building: 16.3s
Building: 16.4s
Building: 16.5s
Building: 16.6s
Building: 16.7s
Building: 16.8s
Building: 17.0s
Building: 17.1s
Building: 17.2s
Building: 17.3s
Building: 17.4s
Building: 17.5s
Building: 17.6s
Building: 17.7s
Building: 17.8s
Building: 17.9s
Building: 18.0s
Building: 18.1s
Building: 18.2s
Building: 18.3s


/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/elementwise_check.hpp: In instantiation of 'void stan::math::elementwise_check(const F&, const char*, const char*, const T&, const char*, const Indexings& ...) [with F = check_nonnegative<Eigen::ArrayWrapper<const Eigen::Matrix<double, -1, 1> > >(const char*, const char*, const Eigen::ArrayWrapper<const Eigen::Matrix<double, -1, 1> >&)::<lambda(double)>; T = Eigen::ArrayWrapper<const Eigen::Matrix<double, -1, 1> >; Indexings = {}; stan::require_eigen_t<S>* <anonymous> = 0; std::enable_if_t<((Eigen::internal::traits<_Rhs>::Flags & Eigen::LinearAccessBit) || T::IsVectorAtCompileTime)>* <anonymous> = 0]':
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/check_nonnegative.hpp:26:20:   required from 'void stan::math::check_nonnegative(const char*, const char*, const T_y&) [with T_y = Eigen::ArrayWrapper<const Eigen::Matrix<double,

Building: 18.4s
Building: 18.5s
Building: 18.6s
Building: 18.7s
Building: 18.8s
Building: 18.9s
Building: 19.0s
Building: 19.1s
Building: 19.2s
Building: 19.3s
Building: 19.4s
Building: 19.5s
Building: 19.6s
Building: 19.7s
Building: 19.8s
Building: 19.9s
Building: 20.0s
Building: 20.1s
Building: 20.2s
Building: 20.3s
Building: 20.4s
Building: 20.6s
Building: 20.7s
Building: 20.8s
Building: 20.9s
Building: 21.0s
Building: 21.1s
Building: 21.2s
Building: 21.3s
Building: 21.4s
Building: 21.5s
Building: 21.6s
Building: 21.7s
Building: 21.8s
Building: 21.9s
Building: 22.0s
Building: 22.1s
Building: 22.2s
Building: 22.3s
Building: 22.4s
Building: 22.5s
Building: 22.6s
Building: 22.7s
Building: 22.8s
Building: 22.9s
Building: 23.0s
Building: 23.1s
Building: 23.2s
Building: 23.3s
Building: 23.4s
Building: 23.5s
Building: 23.6s
Building: 23.7s
Building: 23.8s
Building: 23.9s
Building: 24.0s
Building: 24.1s
Building: 24.2s
Building: 24.4s
Building: 24.5s
Building: 24.6s
Building: 24.7s
Building

Building: 35.5s, done.
Messages from stanc:
Warning in '/tmp/httpstan_ho7nh40v/model_mffguglv.stan', line 9, column 0: Comments
    beginning with # are deprecated and this syntax will be removed in Stan
    2.33.0. Use // to begin line comments; this can be done automatically
    using the auto-format flag to stanc
Warning in '/tmp/httpstan_ho7nh40v/model_mffguglv.stan', line 12, column 0: Comments
    beginning with # are deprecated and this syntax will be removed in Stan
    2.33.0. Use // to begin line comments; this can be done automatically
    using the auto-format flag to stanc


In [18]:
fit = sm.sample(num_chains = n_chains , num_samples = n_samples, num_warmup = n_warmup, stepsize = stepS)

Sampling:   0%
Sampling:   0% (1/6000)
Sampling:   0% (2/6000)
Sampling:   0% (3/6000)
Sampling:   0% (4/6000)
Sampling:   0% (5/6000)
Sampling:   0% (6/6000)
Sampling:   0% (7/6000)
Sampling:   0% (8/6000)
Sampling:   2% (107/6000)
Sampling:   3% (206/6000)
Sampling:   5% (305/6000)
Sampling:   7% (404/6000)
Sampling:   8% (503/6000)
Sampling:  10% (602/6000)
Sampling:  12% (701/6000)
Sampling:  13% (800/6000)
Sampling:  15% (900/6000)
Sampling:  17% (1000/6000)
Sampling:  18% (1100/6000)
Sampling:  20% (1200/6000)
Sampling:  22% (1300/6000)
Sampling:  23% (1400/6000)
Sampling:  24% (1451/6000)
Sampling:  25% (1502/6000)
Sampling:  26% (1553/6000)
Sampling:  27% (1604/6000)
Sampling:  28% (1655/6000)
Sampling:  32% (1905/6000)
Sampling:  34% (2056/6000)
Sampling:  37% (2206/6000)
Sampling:  38% (2305/6000)
Sampling:  47% (2804/6000)
Sampling:  48% (2903/6000)
Sampling:  57% (3402/6000)
Sampling:  58% (3501/6000)
Sampling:  62% (3701/6000)
Sampling:  65% (3900/6000)
Sampling:  72% (430

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.850335, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.901636, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.865711, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.894607, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. c

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.869268, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.878644, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.87957, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.882688, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. co

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.886357, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.918636, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.864488, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.894037, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. c

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.91528, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.860714, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.904418, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.883668, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. co

...

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.879851, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.842934, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.877489, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.88598, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. co

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.932286, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.827318, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.868403, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.870595, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. c

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.881254, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.879776, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.799047, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.848984, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. c

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.865393, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.878092, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.886386, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.833189, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. c

  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.869412, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.867873, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.885182, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. cor_mat(1,1) is 0.870894, but should be near 1.0 (in '/tmp/httpstan_et7t05zc/model_mffguglv.stan', line 59, column 0 to column 40)
  Exception: model_mffguglv_namespace::write_array: cor_mat is not a valid correlation matrix. c

In [13]:
az.summary(fit, var_names = ['beta1', 'beta2', 'rho'])

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta1[0],0.091,0.038,0.023,0.163,0.0,0.0,5976.0,5605.0,1.0
beta1[1],0.313,0.021,0.272,0.351,0.0,0.0,6238.0,5741.0,1.0
beta1[2],-0.377,0.040,-0.450,-0.303,0.0,0.0,8216.0,6101.0,1.0
beta1[3],-0.313,0.034,-0.378,-0.250,0.0,0.0,7578.0,5685.0,1.0
beta1[4],-0.081,0.008,-0.095,-0.067,0.0,0.0,10179.0,6262.0,1.0
beta1[5],0.431,0.035,0.365,0.495,0.0,0.0,7644.0,5980.0,1.0
beta2[0],-0.163,0.030,-0.221,-0.107,0.0,0.0,6981.0,5740.0,1.0
beta2[1],0.238,0.036,0.174,0.309,0.0,0.0,6686.0,5958.0,1.0
"rho[0, 0]",1.000,0.000,1.000,1.000,0.0,0.0,8000.0,8000.0,NaN
"rho[0, 1]",0.000,0.000,0.000,0.000,0.0,0.0,8000.0,8000.0,NaN


In [34]:
iv_reg = IV2SLS.from_formula("lwage_sd ~ 1 + exper_sd + black + south + married + smsa + [educ_sd ~ nearc4 ]", cardNNA).fit()
iv_reg.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               lwage_sd   R-squared:                      0.2513
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2498
No. Observations:                3003   F-statistic:                    892.71
Date:                Fri, Dec 22 2023   P-value (F-stat)                0.0000
Time:                        12:49:50   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0900     0.0406     2.2164     0.0267      0.0104      0.1695
exper_sd       0.5189     0.1855     2.7980     0.0051      0.1554      0.8824
black         -0.2607     0.1117    -2.3343     0.0196     -0.4797     -0.0418
south         -0.2551     0.0517    -4.9314     0.0000     -0.3564     -0.1537
married       -0.0721     0.0114    -6.3037     0.0000     -0.0945     -0.0497
smsa           0.3329     0.0683     4.8721     0.0000      0.1990      0.4668
educ_sd        0.7492     0.2966     2.5258     0.0115      0.1678      1.3306
==============================================================================

Endogenous: educ_sd
Instruments: nearc4
Robust Covariance (Heteroskedastic)
Debiased: False
"""